In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install pyspark
import pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 64.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=d35eb5c9fcf4013a7bbd9a1fea7177bec0285329acd093fbd41ddc17f9db4d60
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext(conf=SparkConf())
spark = SparkSession(sparkContext=sc)

## Logistic regression with pyspark

## Import data

In [4]:
cuse = spark.read.csv('/content/cuse_binary.csv', header=True, inferSchema=True)
cuse.show(5)

+---+---------+---------+---+
|age|education|wantsMore|  y|
+---+---------+---------+---+
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
+---+---------+---------+---+
only showing top 5 rows



## Process categorical columns

The following code does three things with pipeline:

* **`StringIndexer`** all categorical columns
* **`OneHotEncoder`** all categorical index columns
* **`VectorAssembler`** all feature columns into one vector column

### Categorical columns

In [5]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# categorical columns
categorical_columns = cuse.columns[0:3]

### Build StringIndexer stages

In [6]:
stringindexer_stages = [StringIndexer(inputCol=c, 
                                      outputCol='strindexed_' + c) for c in categorical_columns]

# encode label column and add it to stringindexer_stages
stringindexer_stages += [StringIndexer(inputCol='y', 
                                       outputCol='label')]

### Build OneHotEncoder stages

In [7]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, 
                                      outputCol='onehot_' + c) for c in categorical_columns]

### Build VectorAssembler stage

In [8]:
feature_columns = ['onehot_' + c for c in categorical_columns]
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, 
                                        outputCol='features') 

In [11]:
vectorassembler_stage.params

[Param(parent='VectorAssembler_377581eb1615', name='handleInvalid', doc="How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip')."),
 Param(parent='VectorAssembler_377581eb1615', name='inputCols', doc='input column names.'),
 Param(parent='VectorAssembler_377581eb1615', name='outputCol', doc='output column name.')]

### Build pipeline model

In [12]:
# all stages
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

### Fit pipeline model

In [13]:
pipeline_model = pipeline.fit(cuse)

### Transform data

In [14]:
final_columns = feature_columns + ['features', 'label']

cuse_df = pipeline_model.transform(cuse).\
            select(final_columns)
            
cuse_df.show(5)

+-------------+----------------+----------------+-------------------+-----+
|   onehot_age|onehot_education|onehot_wantsMore|           features|label|
+-------------+----------------+----------------+-------------------+-----+
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
+-------------+----------------+----------------+-------------------+-----+
only showing top 5 rows



## Split data into training and test datasets

In [15]:
training, test = cuse_df.randomSplit([0.8, 0.2], seed=1234)

## Build cross-validation model

### Estimator

In [16]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='features', 
                          labelCol='label')

### Parameter grid

In [17]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(logr.regParam, [0, 0.5, 1, 2]).\
    addGrid(logr.elasticNetParam, [0, 0.5, 1]).\
    build()

### Evaluator

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

### Cross-validation model

In [19]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=logr, 
                    estimatorParamMaps=param_grid, 
                    evaluator=evaluator,
                    numFolds=4)

### Fit cross-validation model

In [20]:
cv_model = cv.fit(cuse_df)

### Prediction

In [ ]:
show_columns = ['features', 
                'label', 
                'prediction', 
                'rawPrediction', 
                'probability']

#### Prediction on training data

In [ ]:
pred_training_cv = cv_model.transform(training)
pred_training_cv.select(show_columns).show(5, 
                                           truncate=False)

+---------+-----+----------+------------------------------------------+---------------------------------------+
|features |label|prediction|rawPrediction                             |probability                            |
+---------+-----+----------+------------------------------------------+---------------------------------------+
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
+---------+-----+----------+------------------------------------------+---------------------------------

#### Prediction on test data

In [ ]:
pred_test_cv = cv_model.transform(test)

pred_test_cv.select(show_columns).show(5, 
                                       truncate=False)

+---------+-----+----------+------------------------------------------+---------------------------------------+
|features |label|prediction|rawPrediction                             |probability                            |
+---------+-----+----------+------------------------------------------+---------------------------------------+
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
|(5,[],[])|0.0  |1.0       |[-0.05602447395575516,0.05602447395575516]|[0.4859975438272177,0.5140024561727823]|
+---------+-----+----------+------------------------------------------+---------------------------------

## Intercept and coefficients of the regression model

In [ ]:
print('Intercept: ' + str(cv_model.bestModel.intercept) + "\n"
     'coefficients: ' + str(cv_model.bestModel.coefficients))

Intercept: 0.05602447395575516
coefficients: [-0.28062562258745666,-0.7998574600257556,-1.1892389249150845,0.3249936176955854,-0.8329550151170456]


## Parameters from the best model

In [ ]:
print('The best RegParam is: ', cv_model.bestModel._java_obj.getRegParam(), "\n",
     'The best ElasticNetParam is: cv_model.bestModel._java_obj.getElasticNetParam()')

The best RegParam is:  0.0 
 The best ElasticNetParam is: cv_model.bestModel._java_obj.getElasticNetParam()


# Linear regression with R

## Import Data (below is R code!)

```
#====== This is R code! =========
cuse = read.table('http://data.princeton.edu/wws509/datasets/cuse.dat', header = T)

# convert count data to binary data
not_using = rep(1:nrow(cuse), times=cuse$notUsing)
using = rep(1:nrow(cuse), times=cuse$using)
cuse_binary = cuse[c(not_using, using), 1:3]
cuse_binary$y = c(rep(0, length(not_using)), rep(1, length(using)))

# write data into a file
write.csv(cuse_binary, file='data/cuse_binary.csv', row.names = FALSE)
```

## Process categorical variables 
Process categorical variables so they have the same pattern as in pyspar. Element levels are in the descending order of element frequencies.

```
#====== This is R code! =========
cuse_binary$age = factor(cuse_binary$age, 
                         levels = names(sort(table(cuse_binary$age), decreasing = TRUE)))
cuse_binary$education = factor(cuse_binary$education,
                               levels = names(sort(table(cuse_binary$education), decreasing = TRUE)))
cuse_binary$wantsMore = factor(cuse_binary$wantsMore,
                               levels = names(sort(table(cuse_binary$wantsMore), decreasing = TRUE)))

# encode label column
cuse_binary$y = factor(cuse_binary$y,
                               levels = names(sort(table(cuse_binary$y))))
glm_cuse = glm(y~age + education + wantsMore, data = cuse_binary, family = binomial(link = "logit"))
```

## Results

```
#====== This is R code! =========
glm_cuse$coefficients

 (Intercept)     age25-29       age<25     age40-49 educationlow  wantsMoreno 
   0.7325613    0.5192319    0.9086135   -0.2806254    0.3249947   -0.8329548 
```